In [2]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

import import_ipynb
from DataSet import *
from Neural_Network import SequenceTaggingNet

  0%|          | 0/782 [00:00<?, ?it/s]

importing Jupyter notebook from Neural_Network.ipynb


  0%|          | 0/782 [00:00<?, ?it/s]

0 0.017738264880299864 1.0 1.0


  0%|          | 0/782 [00:00<?, ?it/s]

...

7 3.958789109900028e-06 1.0 1.0


  0%|          | 0/782 [00:00<?, ?it/s]

8 2.606985003811021e-06 1.0 1.0


100%|██████████| 782/782 [02:46<00:00,  4.71it/s]


9 1.732916158973165e-06 1.0 1.0


In [3]:
train_X, train_Y = load_svmlight_file(
    "../05/aclImdb/train/labeledBow.feat"
)
test_X, test_Y = load_svmlight_file(
    "../05/aclImdb/test/labeledBow.feat",
    n_features=train_X.shape[1]
)

In [4]:
model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_Y)
model.score(train_X, train_Y), model.score(test_X, test_Y)

/home/kkw/miniconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.93088, 0.39396)

In [7]:
class SequenceTaggingNet2(SequenceTaggingNet):
    def forward(self, x, h0=None, l=None):
        # Convert ID as a Embedding Mutilple-Dimension-Layer.
        x = self.emb(x)
        
        # Create PckedSequence when it has own length.
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
            x, l, batch_first=True)
        
        # Input into RNN
        x, h = self.lstm(x, h0)
        
        # Input Extracted the Last-Phase only.
        if l is NOne:
            # If you have information of Length of the Last-Phase,
            # YOu can use Inner-Vector-Condition.
            # LSTM usually have not only inner-condition, but also have
            # Block-Shell-Condition, so we are going to use inner-condition
            # only.
            hidden_state, cell_state = h
            x = hidden_state[-1]
        
        else:
            x = x[:, -1, :]
            
        # Input into Linear-Layer
        x = self.linear(x).squeeze()
        return x
    

# Training-Code
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [9]:
from statistics import mean

# num_embedding has a inputed 'train_data.vocab_size + 1' by including 0.
net = SequenceTaggingNet(train_data.vocab_size + 1, num_layers = 2)
net.to("cpu")  # Now we don't have 'cuda:0', so set 'net.to()' gets 'cpu'.
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    
    for x, y, l, in tqdm.tqdm(train_loader):
        # Arrange Length-Array as Length.
        l, sort_idx = torch.sort(l, descending=True)
        # Arrange x,y by used gotten-Index.
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to("cpu")
        y = y.to("cpu")
        
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    
    train_acc = eval_net(net, train_loader, "cpu")
    val_acc = eval_net(net, test_loader, "cpu")
    print(epoch, mean(losses), train_acc, val_acc)

  0%|          | 0/782 [00:00<?, ?it/s]

0 0.021512269319328085 1.0 1.0


  0%|          | 0/782 [00:00<?, ?it/s]

1 0.00015877351448635984 1.0 1.0


  0%|          | 0/782 [00:00<?, ?it/s]

...

7 4.576217617593551e-06 1.0 1.0


  0%|          | 0/782 [00:00<?, ?it/s]

8 2.9533500732972023e-06 1.0 1.0


100%|██████████| 782/782 [02:50<00:00,  4.58it/s]


9 1.9060846161746226e-06 1.0 1.0
